In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('../src'))

import numpy as np
from tqdm.notebook import trange
from tqdm.notebook import tqdm

import model
import model_parameters as MP
#import dask.array as da

#np.get_printoptions()#["threshold"]
#np.set_printoptions(edgeitems=10, linewidth=150)

#np.random.seed(13)

## Mathematical Model

#### Sets
- C	Set of campaigns.
- U	Set of customers.
- H	Set of channels
- D	Set of planning days.
- I	Set of quota categories.
- P	Set of priority categories.


In [2]:
print(f"number of campaigns {MP.C}")
print(f"number of customers {MP.U}")
print(f"number of channels {MP.H}")
print(f"number of planning days {MP.D}")
print(f"number of quota categories {MP.I}")
print(f"number of priority categories {MP.P}")

number of campaigns 10
number of customers 10000
number of channels 3
number of planning days 7
number of quota categories 3
number of priority categories 10


#### Parameters

##### - eligibility
$$
e_{cu}\left\{\begin{array}\\
        1 & \mbox{if }  customer\ u\ is\ eligible\ for\ campaign\ c\\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [3]:
MP.e_cu

array([[0, 0, 0, ..., 0, 1, 1],
       [1, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 1, 1, 1],
       ...,
       [1, 0, 1, ..., 1, 1, 0],
       [1, 1, 0, ..., 1, 1, 0],
       [0, 1, 1, ..., 1, 0, 0]])

##### - quota categories
$$
q_{ic}\left\{\begin{array}\\
        1 & \mbox{if }  campaign\ c\ is\ a\ i^{th} type\ quota\ category\ campaign\ \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [4]:
MP.q_ic

array([[0, 1, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 1, 0, 0, 0, 0, 1],
       [0, 1, 0, 1, 1, 1, 0, 1, 0, 0]])

##### - priority categories
$$r_{cp}: Priority\ value\ of\ campaign\ c\ regarding\ priority\ type\ p\$$

In [5]:
MP.rp_c

array([69, 43, 43, 87, 92, 78, 43, 43, 19, 43])

##### - blokage
$$b: Communication\ limit\ per\ person\ for\ the\ whole\ period\$$

In [6]:
MP.b

7

##### - daily blokage
$$k: Communication\ limit\ per\ person\ at\ each\ day\$$

In [7]:
MP.k

3

##### - campaign blockage
$$l_c: Communication\ limit\ per\ person\ for\ campaign\ c\$$

In [8]:
MP.l_c

array([4, 4, 3, 4, 4, 4, 4, 3, 3, 2])

##### - quota limitations daily/weekly
$$
m_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\
$$
$$
n_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\ each\ day\
$$

In [9]:
(MP.m_i, MP.n_i)

(array([4, 5, 4]), array([1, 2, 2]))

#### - capacity for channel
$$
t_{h,d}: Capacity\ for\ channel\ h\ at\ day\ d.\
$$

In [10]:
MP.t_hd

array([[5000., 6000., 5000., 6000., 6000., 5000., 5000.],
       [5000., 6000., 7000., 6000., 7000., 6000., 6000.],
       [6000., 7000., 7000., 7000., 5000., 7000., 6000.]])

# Model

#### Variables
$$
X_{cuhd}\left\{\begin{array}\\
        1 & \mbox{if } Campaign\ c\ will\ be\ sent\ to\ customer\ u\ through\ Channel\ h\ at\ Day\ d \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

## Maximize
$$\sum_{p \in P}\sum_{c \in C}\sum_{u \in U}\sum_{h \in H}\sum_{d \in D}\,X_{cuhd}\ r_{cp}$$

##### Binary variable (10)
$$
X_{cuhd} \in \{ 1,0 \},\hspace{35pt} \forall c \in C ,\forall u \in U,\forall d \in D, \forall h \in H
$$

In [11]:
X_cuhd = np.zeros((MP.C,MP.U,MP.H,MP.D), dtype='int')

## subject to

##### - eligibility (2)

$$
X_{cuhd}  \leq e_{cu},\hspace{35pt} \forall h \in H,\forall d \in D
$$

In [12]:
MP.eligibility

<function model_parameters.<lambda>(X, c, u, h, d)>

##### - use one channel (3)
$$
\sum_{h}X_{cuhd} \le 1,\hspace{35pt} \forall c \in C \, \forall u \in U,\forall d \in D
$$

In [13]:
MP.one_channel

<function model_parameters.<lambda>(X, c, u, d)>

##### - weekly communication limitation (4)
$$
\sum_{h \in H}\sum_{c \in C}\sum_{d \in D} X_{cuhd}\le b,\hspace{35pt} \forall u \in U
$$

In [14]:
MP.weekly_limitation

<function model_parameters.<lambda>(X, u)>

##### - daily communication limitation (5)
$$
\sum_{h \in H}\sum_{c \in C} X_{cuhd}\le k,\hspace{35pt} \forall u \in U, \forall d \in D
$$

In [15]:
MP.daily_limitation

<function model_parameters.<lambda>(X, u, d)>

##### - campaign communication limit(6)
$$
\sum_{d \in D}\sum_{h \in H} X_{cuhd}\le l_c,\hspace{35pt} \forall c \in C,\forall u \in U;
$$

In [16]:
MP.campaign_limitation

<function model_parameters.<lambda>(X, c, u)>

##### - weekly quota(7)
$$
\sum_{d \in D}\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le m_i,\hspace{35pt} \forall u \in U, \forall i \in I
$$

In [17]:
MP.weekly_quota

<function model_parameters.<lambda>(X, u)>

##### - daily quota(8)
$$
\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le n_i,\hspace{35pt} \forall u \in U,\, \forall d \in D, \forall i \in I
$$

In [18]:
MP.daily_quota

<function model_parameters.<lambda>(X, u, d)>

##### Channel capacity (9)
$$
\sum_{c \in C}\sum_{u \in U}{X_{cuhd}}\le t_{hd},\hspace{35pt} \forall d \in D,\, \forall h \in H
$$

In [19]:
MP.channel_capacity

<function model_parameters.<lambda>(X, h, d)>

In [20]:
#The zero solution
X_cuhd = np.zeros((MP.C,MP.U,MP.H,MP.D), dtype='int')

## Model Definition

In [21]:
c_i = 0
u_i = 1
h_i = 2
d_i = 3

mdl = model.Model([
    model.Constraint('eligibility',MP.eligibility, (c_i, u_i, h_i, d_i,)),
    model.Constraint('channel_capacity',MP.channel_capacity, (h_i, d_i,)),
    model.Constraint('daily_limitation',MP.daily_limitation, (u_i, d_i,)),
    model.Constraint('weekly_limitation',MP.weekly_limitation, (u_i,)),
    model.Constraint('campaign_limitation',MP.campaign_limitation, (c_i, u_i,)),
    model.Constraint('daily_quota',MP.daily_quota, (u_i, d_i,)),
    model.Constraint('one_channel',MP.one_channel, (c_i, u_i, d_i,)),
    model.Constraint('weekly_quota',MP.weekly_quota, (u_i,))
], MP.objective_fn)

### Genetic Algorithm Meta Parameters 

In [22]:
import operator
import functools

INDIV_SIZE = (MP.C,MP.U,MP.H,MP.D)
POP_SIZE = 100#functools.reduce(operator.mul, INDIV_SIZE, 1)
GENERATION_SIZE = 100
CROSSOVER_PROB = 0.6
MUTATION_PROB = 0.01

In [23]:
(INDIV_SIZE, POP_SIZE)

((10, 10000, 3, 7), 100)

### Initialize Population

In [24]:
def fn_initialize_population(population_size, individual_size):
    #TODO take some kernel like matrix for randomization such as e_cu (the eligibility matrix)
    #np.stack([np.stack([MP.e_cu]*3, axis=2)]*7, axis=3)
    overall_size = tuple((population_size,)) + individual_size
    return np.random.randint(2, size=overall_size)

initial_population = fn_initialize_population(POP_SIZE, INDIV_SIZE)

### Fitness

In [25]:
import operator
import functools

def position_gene(j):
    j_d_m = j%MP.D
    j_d_r = j//MP.D
    j_h_m = j_d_r%MP.H
    j_h_r = j_d_r//MP.H
    j_u_m = j_h_r%MP.U
    j_u_r = j_h_r//MP.U
    j_c_m = j_u_r%MP.C
    j_c_r = j_u_r//MP.C
    return (j_c_m,j_u_m,j_h_m,j_d_m)

def fn_fitness_for_indiv(model, indiv):
    dims = indiv.shape
    rng = functools.reduce(operator.mul, dims, 1)
    for j in range(rng):
        if not model.execute(indiv, position_gene(j)):
            return 0
        else:
            return model.calc_value(indiv)
    
def fn_fitness(model, population):
    return [(fn_fitness_for_indiv(model, indiv), indiv) for indiv in population]

population_with_fitness = sorted(fn_fitness(mdl, initial_population), key = lambda tup: tup[0])

### Selection functions

In [26]:
def __random_selection(size):
    return tuple(np.random.randint(low=0, high=POP_SIZE, size=size))

def __spin_wheel(fitnesses, sum_of_fitnesses):
    pin_point = np.random.randint(low=0, high=sum_of_fitnesses)
    fitness_index = 0
    for index, fitness in enumerate(fitnesses):
        pin_point = pin_point + fitness
        if pin_point > sum_of_fitnesses:
            return index
    return index

def roulettewheel_selection(population_with_fitness, size):
    fitnesses = np.array([f for (f,i) in population_with_fitness])
    sum_of_fitnesses = fitnesses.sum()
    if sum_of_fitnesses == 0:
        return __random_selection(size)
    else:
        return tuple([__spin_wheel(fitnesses, sum_of_fitnesses) for i in range(size)])

def __tournament_match(population_with_fitness):
    rivals = __random_selection(size=2)
    return rivals[np.argmax([population_with_fitness[i][0] for i in rivals])]
        
def tournament_selection(population_with_fitness, size):
    return tuple([__tournament_match(population_with_fitness) for i in range(size)])
    
def select(selection_method, size, population_with_fitness, count):
    print(f"selection with {selection_method}")
    for i in range(count):
        yield np.array([population_with_fitness[i][1] for i in selection_method(population_with_fitness, size)])

In [27]:
parents = np.stack([a for a in select(tournament_selection, 2,population_with_fitness, 100)])

selection with <function tournament_selection at 0x7fedff629200>


### Crossover

In [39]:
def __crossover(couple):
    pass
def crossover(population_with_fitness):
    next_gen = population_with_fitness.copy()
    #TODO do-selection
    return next_gen
    
    

In [40]:
crossover(population_with_fitness)

[(0,
  array([[[[0, 1, 0, ..., 1, 1, 0],
           [0, 0, 1, ..., 1, 0, 1],
           [1, 1, 0, ..., 1, 0, 0]],
  
          [[0, 1, 1, ..., 1, 0, 0],
           [0, 1, 1, ..., 1, 0, 1],
           [1, 1, 1, ..., 1, 0, 0]],
  
          [[1, 1, 1, ..., 1, 0, 0],
           [1, 0, 1, ..., 1, 0, 0],
           [1, 0, 0, ..., 1, 0, 0]],
  
          ...,
  
          [[1, 1, 1, ..., 1, 0, 1],
           [0, 0, 0, ..., 0, 0, 0],
           [1, 1, 1, ..., 1, 1, 1]],
  
          [[0, 1, 1, ..., 1, 1, 1],
           [0, 0, 1, ..., 0, 0, 1],
           [1, 1, 1, ..., 1, 0, 1]],
  
          [[1, 1, 1, ..., 0, 0, 0],
           [1, 0, 1, ..., 0, 1, 1],
           [1, 1, 1, ..., 0, 0, 0]]],
  
  
         [[[1, 1, 1, ..., 1, 0, 0],
           [1, 0, 0, ..., 1, 0, 0],
           [1, 1, 1, ..., 0, 0, 1]],
  
          [[0, 0, 0, ..., 0, 0, 1],
           [1, 0, 0, ..., 0, 1, 1],
           [0, 0, 1, ..., 0, 1, 1]],
  
          [[0, 1, 0, ..., 0, 1, 0],
           [1, 1, 1, ..., 1, 0, 0],
     